# Modelling, Analysis and Benchmarking of a Simple Unbalanced LV Network (with Neutral)

## Introduction


<center> <img style="float: middle;" src="../Figures/Figure_1.png" width="40%"> </center>


In [ ]:
import dss
import numpy as np
import roseau.load_flow as rlf

## Open DSS


In [ ]:
# Set up dss_engine
dss_engine = dss.DSS
DSSText = dss_engine.Text
DSSCircuit = dss_engine.ActiveCircuit
DSSSolution = dss_engine.ActiveCircuit.Solution
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue
dss_engine.AllowForms = 0

# Network Modelling - Creating a Circuit
DSSText.Command = "Clear"
DSSText.Command = "Set DefaultBaseFrequency=50"
DSSText.Command = "New Circuit.Simple_LV_Network"
DSSText.Command = "Edit vsource.source bus1=sourceBus basekv=11 pu=1.0 phases=3"

# Adding the 11kV/0.4kV Transformer
DSSText.Command = "New transformer.LVTR Buses=[sourceBus, A.1.2.3.4] Conns=[delta wye] KVs=[11, 0.4] KVAs=[250 250] %Rs=0.00 xhl=2.5 %loadloss=0 "
# DSSText.Command = "dump transformer.LVTR debug"

# Creating the linecodes
DSSText.Command = "new linecode.240sq nphases=4 R1=0.127 X1=0.072 R0=0.342 X0=0.089 units=km"
DSSText.Command = "new linecode.16sq nphases=2 R1=1.15 X1=0.083 R0=1.2 X0=0.083 units=km"

# Creating the 400V and 230V lines
DSSText.Command = "new line.A_B bus1=A.1.2.3.4 bus2=B.1.2.3.4 length=1 phases=4 units=km linecode=240sq"
DSSText.Command = "new line.B_L1 bus1=B.1.4 bus2=C.1.2 length=0.01 phases=2 units=km linecode=16sq"
DSSText.Command = "new line.B_L2 bus1=B.2.4 bus2=D.1.2 length=0.01 phases=2 units=km linecode=16sq"
DSSText.Command = "new line.B_L3 bus1=B.3.4 bus2=E.1.2 length=0.01 phases=2 units=km linecode=16sq"
# DSSText.Command = "dump linecode.240sq debug"
# DSSText.Command = "dump line.A_B debug"

# Connecting loads to a bus
# Note: I (Ali) couldn't make it work with phases=2 for loads. It will need more digging
# into the OpenDSS model to understand the discrepancy between our phase-neutral load
# definition and theirs. For the time being, keeping phases=1 works fine as long as
# the correct bus phases are used in the load definition
DSSText.Command = "new load.Load_1 bus1=C.1.2 phases=1 kV=(0.4 3 sqrt /) kW=7 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_2 bus1=D.1.2 phases=1 kV=(0.4 3 sqrt /) kW=6 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_3 bus1=E.1.2 phases=1 kV=(0.4 3 sqrt /) kW=8 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"

# Set the Control mode and the Voltage bases
DSSText.Command = "set controlmode=static"
DSSText.Command = "set mode=snapshot"
DSSText.Command = "Set VoltageBases=[11 0.4]"
DSSText.Command = "calcvoltagebases"

In [ ]:
# Run the power flow simulation
DSSSolution.Solve()
if DSSSolution.Converged:
    print("The Circuit was Successfully Solved")
else:
    raise RuntimeError("DID NOT CONVERGE")

In [ ]:
# Extract the results
for active_load in ("Load_1", "Load_2", "Load_3"):
    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    print(f"{active_load}:  ")
    pp, pn = DSSCircuit.ActiveElement.Powers[0::2]
    qp, qn = DSSCircuit.ActiveElement.Powers[1::2]
    print(f"Active power demand (P)= {round(pp+pn, 3)} kW")
    print(f"Reactive power demand (Q)= {round(qp+qn, 3)} kvar")
print()

for active_load in ("Load_1", "Load_2", "Load_3"):
    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    bus_name = DSSCircuit.ActiveElement.Properties("bus1").Val
    DSSCircuit.SetActiveBus(bus_name)
    voltages = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
    (v1,) = np.abs(voltages[:-1] - voltages[-1])
    print(f"The voltage [in p.u.] of the bus connected to {active_load} = {round(v1, 3)} pu")
print()

active_bus = "A"
DSSCircuit.SetActiveBus(active_bus)
print(f"Voltage magnitudes at bus {active_bus}:  ")
voltages = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
v1, v2, v3 = np.abs(voltages[:-1] - voltages[-1])
print(f"Voltage magnitude - phase 1 = {round(v1, 3)} pu")
print(f"Voltage magnitude - phase 2 = {round(v2, 3)} pu")
print(f"Voltage magnitude - phase 3 = {round(v3, 3)} pu")
print()

DSSCircuit.SetActiveElement("transformer.LVTR")
print("Results of the transformer LVTR: ")
# Indices [0,4[ are primary powers, indices [4, 8[ are secondary powers
transformer_p = DSSCircuit.ActiveElement.Powers[0::2]
transformer_q = DSSCircuit.ActiveElement.Powers[1::2]
print(f"Active power (P) supplied to phase 1 = {round(abs(transformer_p[4]), 5)} kW")
print(f"Active power (P) supplied to phase 2 = {round(abs(transformer_p[5]), 5)} kW")
print(f"Active power (P) supplied to phase 3 = {round(abs(transformer_p[6]), 5)} kW")
print(f"Reactive power (Q) supplied to phase 1 = {round(abs(transformer_q[4]), 5)} kvar")
print(f"Reactive power (Q) supplied to phase 2 = {round(abs(transformer_q[5]), 5)} kvar")
print(f"Reactive power (Q) supplied to phase 3 = {round(abs(transformer_q[6]), 5)} kvar")
print(f"Total active power (P) losses = {round(sum(transformer_p), 5)} kW")
print(f"Total reactive power (P) losses = {round(sum(transformer_q), 5)} kvar")
print()

line_loss = DSSCircuit.LineLosses
lines_p, lines_q = line_loss
print("Results of the Power Losses: ")
print(f"Active power (P) losses = {round(abs(lines_p), 3)} kW")
print(f"Reactive power (Q) losses = {round(abs(lines_q), 3)} kvar")

## Roseau Load Flow


In [ ]:
# Buses
# -----
source_bus = rlf.Bus(id="source_bus", phases="abc")
bus_a = rlf.Bus(id="bus_A", phases="abcn")
bus_b = rlf.Bus(id="bus_B", phases="abcn")
bus_c = rlf.Bus(id="bus_C", phases="an")
bus_d = rlf.Bus(id="bus_D", phases="bn")
bus_e = rlf.Bus(id="bus_E", phases="cn")

# References
# ----------
pref_mv = rlf.PotentialRef(id="pref_mv", element=source_bus)
ground = rlf.Ground(id="ground")
ground.connect(bus=bus_a, phase="n")
pref_lv = rlf.PotentialRef(id="pref_lv", element=ground)

# Sources
# -------
un_mv = 11_000
un_lv = 400
voltages = un_mv * np.exp([0, -2j * np.pi / 3, 2j * np.pi / 3])
vs = rlf.VoltageSource(id="vs", bus=source_bus, voltages=voltages)

# Transformers
# ------------
tp = rlf.TransformerParameters.from_open_dss(
    id="LVTR",
    conns=("delta", "wye"),
    kvs=rlf.Q_([un_mv, un_lv], "V"),
    kvas=(250, 250),
    leadlag="euro",  # <- should be "ansi" (i.e. "Dyn1") but we don't have this in RLF yet
    xhl=2.5,
    rs=0,
    loadloss=0,
    noloadloss=0,
    imag=0,
    normhkva=None,
)
transformer = rlf.Transformer("LVTR", bus1=source_bus, bus2=bus_a, parameters=tp)

# Lines
# -----
lp_240 = rlf.LineParameters.from_open_dss(
    id="linecode-240sq",
    nphases=4,
    r1=rlf.Q_(0.127, "ohm/km"),
    x1=rlf.Q_(0.072, "ohm/km"),
    r0=rlf.Q_(0.342, "ohm/km"),
    x0=rlf.Q_(0.089, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)
lp_16 = rlf.LineParameters.from_open_dss(
    id="linecode-16sq",
    nphases=2,
    r1=rlf.Q_(1.150, "ohm/km"),
    x1=rlf.Q_(0.083, "ohm/km"),
    r0=rlf.Q_(1.200, "ohm/km"),
    x0=rlf.Q_(0.083, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)

line_ab = rlf.Line(
    "lineA_B", bus1=bus_a, bus2=bus_b, phases="abcn", parameters=lp_240, length=rlf.Q_(1, "km"), ground=ground
)
line_bc = rlf.Line(
    "lineB_C", bus1=bus_b, bus2=bus_c, phases="an", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)
line_bd = rlf.Line(
    "lineB_D", bus1=bus_b, bus2=bus_d, phases="bn", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)
line_be = rlf.Line(
    "lineB_E", bus1=bus_b, bus2=bus_e, phases="cn", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)


# Loads
# -----
def complex_power(p: float, pf: float) -> complex:
    phi = np.arccos(pf)
    q = p * np.tan(phi)
    return p + 1j * q


load1 = rlf.PowerLoad(id="load1", bus=bus_c, phases="an", powers=[complex_power(7e3, 0.95)])
load2 = rlf.PowerLoad(id="load2", bus=bus_d, phases="bn", powers=[complex_power(6e3, 0.95)])
load3 = rlf.PowerLoad(id="load3", bus=bus_e, phases="cn", powers=[complex_power(8e3, 0.95)])

en = rlf.ElectricalNetwork.from_element(initial_bus=source_bus)
en

In [ ]:
en.solve_load_flow()

In [ ]:
for load in (load1, load2, load3):
    print(f"{load.id}:  ")
    load_powers = load.res_powers.m_as("kVA").sum()
    print(f"Active power demand (P)= {round(load_powers.real, 3)} kW")
    print(f"Reactive power demand (Q)= {round(load_powers.imag, 3)} kvar")
print()

for load in (load1, load2, load3):
    voltages_pu = abs(load.res_voltages.m[0]) / (un_lv / np.sqrt(3))
    print(f"The voltage [in p.u.] of the bus connected to {load.id} = {round(voltages_pu, 3)} pu")
print()

print("Voltage magnitudes at bus A:  ")
bus_a_voltages_pu = abs(bus_a.res_voltages.m) / (un_lv / np.sqrt(3))
print(f"Voltage magnitude - phase 1 = {round(bus_a_voltages_pu[0], 3)} pu")
print(f"Voltage magnitude - phase 2 = {round(bus_a_voltages_pu[1], 3)} pu")
print(f"Voltage magnitude - phase 3 = {round(bus_a_voltages_pu[2], 3)} pu")
print()

transformer_powers = transformer.res_powers[1].m_as("kVA")
transformer_power_losses = transformer.res_power_losses.m_as("kVA")
print("Results of the transformer LVTR: ")
print(f"Active power (P) supplied to phase 1 = {round(abs(transformer_powers[0].real), 5)} kW")
print(f"Active power (P) supplied to phase 2 = {round(abs(transformer_powers[1].real), 5)} kW")
print(f"Active power (P) supplied to phase 3 = {round(abs(transformer_powers[2].real), 5)} kW")
print(f"Reactive power (Q) supplied to phase 1 = {round(abs(transformer_powers[0].imag), 5)} kvar")
print(f"Reactive power (Q) supplied to phase 2 = {round(abs(transformer_powers[1].imag), 5)} kvar")
print(f"Reactive power (Q) supplied to phase 3 = {round(abs(transformer_powers[2].imag), 5)} kvar")
print(f"Total active power (P) losses = {round(transformer_power_losses.real, 5)} kW")
print(f"Total reactive power (P) losses = {round(transformer_power_losses.imag, 5)} kvar")
print()

total_line_loss = sum(br.res_power_losses.m_as("kVA").sum() for br in en.branches.values() if isinstance(br, rlf.Line))
print("Results of the Power Losses: ")
print(f"Active power (P) losses = {round(abs(total_line_loss.real), 3)} kW")
print(f"Reactive power (Q) losses = {round(abs(total_line_loss.imag), 3)} kvar")